# Challenge 2

In this challenge we will continue working with `Pokemon.csv`. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemons are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.

## Provlem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# import libraries
import numpy as np
import pandas as pd

In [46]:
# importing the dataset and inspect the first few rows of the data
pk = pd.read_csv('Pokemon.csv')
pk.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [47]:
print(len(pk))

800


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemons?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
pk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [4]:
# your code here
pk['total_formula'] = pk['HP'] + pk['Attack'] + pk['Defense'] + pk['Sp. Atk'] + pk['Sp. Def'] + pk['Speed']
pk[['Total','total_formula']].head(10)


,Total,total_formula
0,318,318
1,405,405
2,525,525
3,625,625
4,309,309
5,405,405
6,534,534
7,634,634
8,634,634
9,314,314


## Provlem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](one-hot-encoding.png)

In [57]:
cols_names= ['Grass','Poison','Fire','Flying','Dragon','Water','Bug','Normal','Electric','Ground','Fairy','Fighting','Psychic','Rock','Ice','Ghost','Dark']

def is_nan(x):
    if pd.isna(x['Type 2']):
        return ''
    else:
        return '-'+x['Type 2']
combo_type1 = pk['Type 1']
combo_type2 = pk.apply(lambda x: is_nan(x), axis=1)
pk['Type'] = combo_type1 + combo_type2
pk['Type'].head(20)

0     Grass-Poison
1     Grass-Poison
2     Grass-Poison
3     Grass-Poison
4             Fire
5             Fire
6      Fire-Flying
7      Fire-Dragon
8      Fire-Flying
9            Water
10           Water
11           Water
12           Water
13             Bug
14             Bug
15      Bug-Flying
16      Bug-Poison
17      Bug-Poison
18      Bug-Poison
19      Bug-Poison
Name: Type, dtype: object

In [44]:
pk1 = pd.get_dummies(pk['Type 1'])
pk2 = pd.get_dummies(pk['Type 2'])


800
800


In [78]:
cols = ['Grass','Poison','Fire','Flying','Dragon','Water','Bug','Normal','Electric','Ground','Fairy','Fighting','Psychic','Rock','Ice','Ghost','Dark']
for i in range(len(cols)):
    col_name = cols[i]
    pk1[col_name] = np.where(pk2[col_name] == 1, pk2[col_name], pk1[col_name])
pk1.head(10)


,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
7,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [79]:
pk3 = pd.concat([pk['Type'],pk1], axis=1)
pk4 = pd.merge(pk, pk3, on='Type')

In [77]:
pk4[['Name','Type 1', 'Type 2','Total','Grass','Poison','Fire','Flying','Dragon','Water','Bug','Normal','Electric','Ground','Fairy','Fighting','Psychic','Rock','Ice','Ghost','Dark']].sort_values(by='Total', ascending=False).head(20)

,Name,Type 1,Type 2,Total,Grass,Poison,Fire,Flying,Dragon,Water,...,Normal,Electric,Ground,Fairy,Fighting,Psychic,Rock,Ice,Ghost,Dark
11555,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11575,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11573,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11572,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11571,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11570,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11569,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11568,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11567,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11565,MewtwoMega Mewtwo Y,Psychic,NaN,780,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


## Provlem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [ ]:
# your code here


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [80]:
# your code here
pk[['Name','Type 1','Type 2','Total']].sort_values(by='Total', ascending=False).head(1)

,Name,Type 1,Type 2,Total
426,RayquazaMega Rayquaza,Dragon,Flying,780
